<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
#from sklearn.preprocessing import RobustScaler, StandardScaler
#from sklearn.pipeline import Pipeline
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
#from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score, roc_curve
from sklearn.utils import shuffle
#import matplotlib.pyplot as plt
import warnings
#from tqdm import tqdm
warnings.filterwarnings('ignore')
#from lightgbm import LGBMClassifier
#import lightgbm as lgb

In [2]:
toxic_comments = pd.read_csv('/datasets/toxic_comments.csv')
toxic_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [3]:
toxic_comments.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
corpus = list(toxic_comments['text'])

In [5]:
#Посмотрим сколько у нас токсичных/нектоксичных текстов
display(toxic_comments['toxic'].value_counts())
#Выведем соотношение
class_ratio = toxic_comments['toxic'].value_counts()[0] / toxic_comments['toxic'].value_counts()[1]
class_ratio

0    143106
1     16186
Name: toxic, dtype: int64

8.841344371679229

Вывод

Видим что классы несбалансированы, применем несколько способов балансировки и сравним их

In [6]:
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    text = text.lower()
    lemm_text = " ".join(lemmatizer.lemmatize(word) for word in text.split())
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text) 
    return " ".join(cleared_text.split())

toxic_comments['lemm_text'] = toxic_comments['text'].apply(lemmatize_text)

toxic_comments = toxic_comments.drop(['text'], axis=1)


In [7]:
toxic_comments['lemm_text'][3]

'more i can t make any real suggestion on improvement i wondered if the section statistic should be later on or a subsection of types of accidents i think the reference may need tidying so that they are all in the exact same format ie date format etc i can do that later on if no one else doe first if you have any preference for formatting style on reference or want to do it yourself please let me know there appears to be a backlog on article for review so i guess there may be a delay until a reviewer turn up it s listed in the relevant form eg wikipedia good article nominations transport'

In [8]:
dataset = pd.DataFrame({'lemma': corpus, 'toxic': toxic_comments['toxic']})

In [9]:
# Разобьем выборку по отношению 50/20/20. Уменьшим количество кроссвалидаций до 3 из-за размера выборки.
target = toxic_comments['toxic']
features = toxic_comments.drop(['toxic'], axis=1)

features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target, 
                                                                              test_size=0.5, 
                                                                              random_state=1515)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
                                                                            target_valid, 
                                                                            test_size=0.5,
                                                                            random_state=1515)



nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

features_train = count_tf_idf.fit_transform(features_train['lemm_text'].values)
features_valid = count_tf_idf.transform(features_valid['lemm_text'].values)
features_test = count_tf_idf.transform(features_test['lemm_text'].values)
print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)
cv_counts = 2

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(79646, 110921)
(39823, 110921)
(39823, 110921)


In [10]:
classificator = LogisticRegression()
train_f1 = cross_val_score(classificator, 
                      features_train, 
                      target_train, 
                      cv=cv_counts, 
                      scoring='f1').mean()
print('F1 на CV', train_f1)

F1 на CV 0.637618679285346


Балансировка данных

Уменьшение размера весов классов

In [11]:
dict_classes={0:1, 1:class_ratio}
classificator = LogisticRegression(class_weight=dict_classes)
train_f1_ballanced = cross_val_score(classificator, 
                                    features_train, 
                                    target_train, 
                                    cv=cv_counts, 
                                    scoring='f1').mean()
print('F1 на CV с балансированными классами', train_f1_ballanced)

F1 на CV с балансированными классами 0.7456315886644493


In [12]:
classificator = LogisticRegression(class_weight='balanced')
train_f1_balanced = cross_val_score(classificator, 
                                    features_train, 
                                    target_train, 
                                    cv=cv_counts, 
                                    scoring='f1').mean()
print('F1 на CV с балансированными классами', train_f1_ballanced)

F1 на CV с балансированными классами 0.7456315886644493


Ресемплинг с уменшением класса 0

In [13]:
toxic_comments_train = toxic_comments.iloc[target_train.index]

target_train_class_zero = toxic_comments_train[toxic_comments_train['toxic'] == 0]['toxic']
target_train_class_one = toxic_comments_train[toxic_comments_train['toxic'] == 1]['toxic']

In [14]:
target_train_class_zero_downsample = target_train_class_zero.sample(target_train_class_one.shape[0],
                                                                    random_state=12082020)
target_train_downsample = pd.concat([target_train_class_zero_downsample, target_train_class_one])

features_train_downsample = toxic_comments.iloc[target_train_downsample.index]
features_train_downsample, target_train_downsample = shuffle(features_train_downsample,
                                                             target_train_downsample,
                                                             random_state=12082020)
features_train_downsample = count_tf_idf.transform(features_train_downsample['lemm_text']
                                                   .values.astype('U'))

In [15]:
classificator = LogisticRegression()
train_f1_downsampled = cross_val_score(classificator,
                      features_train_downsample, 
                      target_train_downsample, 
                      cv=cv_counts, 
                      scoring='f1').mean()
print('F1 на CV с уменьшением классов', train_f1_downsampled)

F1 на CV с уменьшением классов 0.874815273093978


Видим значительный прирост f1 меры

Вывод

В ходе исследования обнаружил большой разболанс. Выборка была разбита на тестовую, валидационную и тестовую. Самого текста была проведена лемматизация

## Обучение

Начнём с логистической регрессии

In [16]:
%%time

# чтобы не перебирать вручную гиперпараметры, используем метод GridSearchCV для автоматического подбора
model = LogisticRegression()
giperparametr = [{'solver':['newton-cg', 'lbfgs', 'liblinear'], 'C':[0.1, 1, 10], 'class_weight':[dict_classes]}]
clf = GridSearchCV(model, giperparametr, scoring='f1',cv=cv_counts)
clf.fit(features_train, target_train)
display(clf.best_params_)

{'C': 10, 'class_weight': {0: 1, 1: 8.841344371679229}, 'solver': 'liblinear'}

CPU times: user 1min 28s, sys: 3min 5s, total: 4min 34s
Wall time: 4min 34s


расчитаем f1

In [17]:
%%time

LR_best_params = clf.best_params_
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
cv_f1_LR = max(means)
model = LogisticRegression()
model.set_params(**LR_best_params)
model.fit(features_train, target_train)

CPU times: user 7.73 s, sys: 13.7 s, total: 21.4 s
Wall time: 21.4 s


LogisticRegression(C=10, class_weight={0: 1, 1: 8.841344371679229},
                   solver='liblinear')

In [18]:
%%time
target_predict = model.predict(features_valid)

CPU times: user 3.53 ms, sys: 3.35 ms, total: 6.88 ms
Wall time: 6.69 ms


In [19]:
%%time

valid_f1_LR = f1_score(target_valid, target_predict)
print('F1 на cv', cv_f1_LR)
print('F1 на валидации', valid_f1_LR)

F1 на cv 0.7502782510030439
F1 на валидации 0.7544441810855652
CPU times: user 15.2 ms, sys: 83 µs, total: 15.3 ms
Wall time: 53.1 ms


Результат очень неплохой!

Далее попробуем CatBoostClassifier

In [20]:
%%time

model = CatBoostClassifier(verbose=False, iterations=250)
model.fit(features_train, target_train)
target_predict = model.predict(features_valid)

CPU times: user 6min 51s, sys: 12 s, total: 7min 3s
Wall time: 7min 5s


In [21]:
%%time
target_predict = model.predict(features_valid)

CPU times: user 388 ms, sys: 8.06 ms, total: 396 ms
Wall time: 412 ms


In [22]:
%%time

cv_f1_CBC = cross_val_score(model,
                                         features_train, 
                                         target_train, 
                                         cv=cv_counts, 
                                         scoring='f1').mean()
valid_f1_CBC = f1_score(target_valid, target_predict)
print('F1 на cv', cv_f1_CBC)
print('F1 на валидации', valid_f1_CBC)

F1 на cv 0.7121617499476107
F1 на валидации 0.728860427885315
CPU times: user 7min 50s, sys: 7.41 s, total: 7min 57s
Wall time: 7min 59s


Вывод: лучше всего себя показала логистическая регрессия, её и будем использовать. Но если бустинг обучать дольше, то она может показать ещё лучшие результаты.

## Выводы

In [23]:
# хочу посмотреть на результат в разных метриках
classificator = LogisticRegression()
classificator.set_params(**LR_best_params)
classificator.fit(features_train, target_train)
predict_test = classificator.predict(features_test)
print('Метрики LogisticRegression')
print('F1:', f1_score(target_test, predict_test))
print('Precision:', precision_score(target_test, predict_test))
print('Recall:', recall_score(target_test, predict_test))
print('Accuracy:', accuracy_score(target_test, predict_test))
print()

Метрики LogisticRegression
F1: 0.762859529776823
Precision: 0.7370848708487084
Recall: 0.7905021023992085
Accuracy: 0.9501042111342691



Результаты, конечно, отличаются очень сильно. Как понять какую метрику выбрать для проверки? У них есть какие-то строгие распределния?

Вывод

Нашей целью было найти и обучить модель для магазина "ВИКИШОП", которая способна классифицировать комментарии на позитивные и негативные для того, чтобы в дальнейшем токсичные отзывы отправлялись на модерацию. При этом достигуть результата предсказания наилучшей модели по мерке f1 не меньше 0.75. В итоге она получилась даже чуточку лучше- 0,76